Importação e Leitura dos Dados

In [ ]:
import pandas as pd
from datetime import datetime
import os
import sys
import calendar

In [ ]:
df = pd.read_csv('../data/raw/superstore_final_dataset.csv', encoding='latin1')

Entendendo a base de Dados

Nessa sessão será executado alguns comandos, para entender melhor sobre a estrutura e como funciona o dataset

In [ ]:
df.head(10)

In [ ]:
df.info() # Entendendo melhor como funciona e está estruturado os dados

In [ ]:
df.columns # Verificando quais são as colunas existentes nesse dataset

In [ ]:
df.describe()

Preparação dos Dados

Nessa sessão será executado as principais formatações, limpezas e preparações dos dados para as análise. Garantindo a consistência e clareza dos dados

In [ ]:
df_copy = df.copy()
df_copy = df_copy.set_index('Row_ID')

In [ ]:
df_copy.isnull().sum()
mask_index_nan = df_copy['Postal_Code'].isnull() 
# Uma máscara com os index onde o Postal_Code está com o valor NaN, para ter um controle exato de onde essses valores estão

In [ ]:
# Mudar os valores NaN do Postal_Code para o valor 0
df_copy.loc[df_copy['Postal_Code'].isnull(), 'Postal_Code'] = 0
df_copy[mask_index_nan] # Verificar se a correção foi feita corretamente, se sim não deve retornar nenhuma linha

In [ ]:
df_copy.loc[df_copy.duplicated()]
# Ter o controle e visualização de qual linha realmente é a duplicada, somente para uma breve visualização das linhas
linhas_duplicadas = df_copy['Order_ID'] == 'US-2015-150119'

In [ ]:
# Remover as duplicatas
mask = ~df_copy.duplicated(keep='first') # Uso do operador not, para que as duplicatas sejam false, e ao aplicar a máscara no df, ela não apareça, pois é false
df_copy = df_copy[mask].copy()

In [ ]:
# Função para formatar as datas
def formatar_datas(valor):
    date = valor.split('/')
    nova_data = f'{date[2]}-{date[1]}-{date[0]}'
    return nova_data
df_copy['Order_Date'] = df_copy['Order_Date'].apply(formatar_datas)
df_copy['Ship_Date'] = df_copy['Ship_Date'].apply(formatar_datas)

In [ ]:
# Converter Order_Date e Ship_Date para o tipo datetime
df_copy['Order_Date'] = pd.to_datetime(df_copy['Order_Date'], errors='coerce')
df_copy['Ship_Date'] = pd.to_datetime(df_copy['Ship_Date'], errors='coerce')

Analisar o total de vendas entre 2015-2019

Nessa sessão será extraído o valor total de arrecadação do supermercado nos anos de 2015 a 2019

In [ ]:
total_vendas = df_copy['Sales'].sum()
total_vendas

Analisar o total de vendas por ano entre 2015-2019

Nessa sessão será extraído a arrecadação do supermercado em cada ano individual

In [ ]:
# Separar cada ano
vendas_por_ano = {} # Local onde a quantidade de vendas de cada ano será armazenado
def soma_vendas_por_ano(valor):
    for x in range(5, 10):
        ano = int(f'201{x}')
        query = round(df_copy.loc[df_copy['Ship_Date'].dt.year == ano, 'Sales'].sum(), 2)
        valor.update({f'201{x}':float(query)})
    series = pd.Series(valor)
    return pd.DataFrame(series, columns=['Total de Vendas']).sort_values(by='Total de Vendas', ascending=False)
soma_vendas_por_ano(vendas_por_ano)

Identificação Mensal dos Picos e Quedas de Vendas Durante o Ano

Nessão sessão será retornado os valores de vendas em cada mês entre os anos de 2015-2019

In [ ]:
vendas_por_mes = {} # Local onde a quantidade de vendas por mês ficará armazenado

# Função pegar a soma de vendas por mês em cada ano
def soma_vendas_por_mes(valor):
    for x in range(5,10):
        ano = int(f'201{x}')
        df_ano_filtrado = df_copy.loc[df_copy['Ship_Date'].dt.year == ano]
        for m in range(1, 13):
            
            ultimo_dia = calendar.monthrange(ano, m)[1]
            data_inicial = datetime.strptime(f'201{x}-{m:02d}-01', '%Y-%m-%d')
            data_final = datetime.strptime(f'201{x}-{m:02d}-{ultimo_dia}', '%Y-%m-%d')
            
            valor_vendas_mes = round(df_ano_filtrado.loc[(df_ano_filtrado['Ship_Date'] >= data_inicial) & (df_ano_filtrado['Ship_Date'] <= data_final), 'Sales'].sum(), 2)
            
            if ano not in valor:
                valor.update({ano: {
                    f'mes_{m}':  f'$ {float(valor_vendas_mes)}'
                }})
            else:
               valor[ano].update({f'mes_{m}': f'$ {float(valor_vendas_mes)}'})
    return valor
soma_vendas_por_mes(vendas_por_mes)

Analisar vendas por Categoria e Sub-categoria de produtos

In [ ]:
# Criar o agrupamento por Categoria e Sub-Categoria para tirar a quantidade de vendas por produto (individual)
vendas_por_prod = df_copy.groupby(['Category', 'Sub_Category'])['Sales'].sum()
vendas_por_prod = pd.DataFrame(vendas_por_prod)

In [ ]:
# Criar um grupo com Categoria, para tirar a quantidade de vendas por categoria
vendas_por_categoria = df_copy.groupby('Category')['Sales'].sum()
vendas_por_categoria = pd.DataFrame(vendas_por_categoria).sort_values(by='Sales', ascending=False)

Analisar vendas por região

In [ ]:
vendas_por_regiao = df_copy.groupby(['Region', 'State'])['Sales'].sum()
vendas_por_regiao = pd.DataFrame(vendas_por_regiao)

Analisar o comportamento do cliente

In [52]:
df_copy.head(2)

,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales
Row_ID,,,,,,,,,,,,,,,,,
1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96
2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94
